# Tides acting on a post- natal kick binary

Study of the impact of tides on non-coplanar binaries

In this notebook we show the results of our code for the evolution of tides acting on a HMXB using different values for the inclination just after a momentum kick

### load modules

In [ ]:
# auto load modules, no need to restart kernel
%load_ext autoreload
%autoreload 2

from matplotlib.pyplot import cm
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import MultipleLocator

# custom module to read MESA output
from mesa_reader import *

### utility functions

#### 1. group_consecutives

Group lists by consecutive numbers. Useful when having several mass-transfer (MT) phases

#### 2. find_tams_age

Locate TAMS point and the age of the star at that point

In [ ]:
def group_consecutives(vals, step=1):
    """Return list of consecutive lists of numbers from vals (number list)"""
    run = []
    result = [run]
    expect = None
    for v in vals:
        if (v == expect) or (expect is None):
            run.append(int(v))
        else:
            run = [int(v)]
            result.append(run)
        expect = int(v) + step
    return result


def find_tams_age(center_h1, age):
    """Find the value of the terminal age main sequence (TAMS)"""
    for k, h1 in enumerate(center_h1):
        if h1 < 1e-4:
            return age[k]

#### MESA output as dictionaries

In [ ]:
# dictionaries with MESA output plus the location of the TAMS for the star
t_tams = dict()
binary, companion = dict(), dict()

---

#### inclinations used

In [ ]:
inclinations = (0, 20, 40, 60, 80, 100, 120, 140, 160)

# hex color from https://coolors.co/palettes/popular/gradient
colorbar = (
    "#000000",
    "#012A4A",
    "#013A63",
    "#01497C",
    "#014F86",
    "#2A6F97",
    "#2C7DA0",
    "#468FAF",
    "#61A5C2",
    "#89C2D9",
    "#A9D6E5",
)

colorbar_appendix = (
    "#123A6C",
    "#61AEAD",
    "#DEBA93",
    "#9E3253",
)

#### load data for high spin case with ST dynamo

In [ ]:
root = "../data/raw/asb2022/caseA/high_spin/with_ST"

binary["high_spin_with_ST"] = dict()
companion["high_spin_with_ST"] = dict()
t_tams["high_spin_with_ST"] = dict()
for inclination in inclinations:
    companion["high_spin_with_ST"][inclination] = MesaInfo(
        f"{root}/LOGS_{inclination}/history.data"
    )
    binary["high_spin_with_ST"][inclination] = MesaInfo(
        f"{root}/binary_history_{inclination}.data"
    )
    t_tams["high_spin_with_ST"][inclination] = find_tams_age(
        companion["high_spin_with_ST"][inclination].data.center_h1,
        companion["high_spin_with_ST"][inclination].data.star_age,
    )
    print(f"{root}/LOGS_{inclination}/history.data ... loaded")

In [ ]:
# find when there is a first MT phase in the evolution of the binary
i = 0  # inclination
mask = binary["high_spin_with_ST"][i].data.lg_mtransfer_rate > -16
init_MT = (
    binary["high_spin_with_ST"][i].data.age[mask][0] / t_tams["high_spin_with_ST"][i]
)

## plots

### evolution of orbital parameters

### - eccentricity and orbital periods

In [ ]:
# fig configs
plt.style.use("../config/paper-style.mpl")
fig, axs = plt.subplots(figsize=(4.5, 4), nrows=2, sharex=True)
plt.subplots_adjust(hspace=0)

# second axis config
left, bottom, width, height = [0.22, 0.75, 0.3, 0.17]
ax2 = fig.add_axes([left, bottom, width, height])

# axis labels
latex = "{\\rm TAMS}"
axs[-1].set_xlabel(f"$t/t_{latex}$")
latex = "{\\rm orb}"
axs[0].set_ylabel(f"$P_{latex}$ [days]")
axs[1].set_ylabel("eccentricity")

# plots !
for k, inclination in enumerate(inclinations):
    data_null = binary["high_spin_with_ST"][inclination]
    tams_age = t_tams["high_spin_with_ST"][inclination]

    axs[0].plot(
        data_null.data.age / tams_age, data_null.data.period_days, color=colorbar[k]
    )

    ax2.plot(
        data_null.data.age / tams_age, data_null.data.period_days, color=colorbar[k]
    )

    axs[1].plot(
        data_null.data.age / tams_age,
        data_null.data.eccentricity,
        color=colorbar[k],
        label=f"{inclination} deg",
    )

# ticks params & limits
axs[0].set_yticks(np.arange(2, 15, 1))
axs[0].yaxis.set_minor_locator(MultipleLocator(0.5))
axs[0].set_ylim([1.5, 6.5])
axs[1].set_yticks(np.arange(0, 1.2, 0.1))
axs[1].yaxis.set_minor_locator(MultipleLocator(0.05))
axs[1].set_ylim([0, 0.33])
axs[-1].set_xticks(np.arange(0, 1.1, 0.1))
axs[-1].xaxis.set_minor_locator(MultipleLocator(0.05))
axs[-1].set_xlim([0, 1])
ax2.set_xticks(np.arange(0, 1, 0.1))
ax2.xaxis.set_minor_locator(MultipleLocator(0.05))
ax2.set_xlim([0, 0.5])
ax2.set_yticks(np.arange(2, 4, 0.1))
ax2.yaxis.set_minor_locator(MultipleLocator(0.05))
ax2.set_ylim([2.85, 3.25])

# reduce fontsize for inset plot
for item in (
    [ax2.title, ax2.xaxis.label, ax2.yaxis.label]
    + ax2.get_xticklabels()
    + ax2.get_yticklabels()
):
    item.set_fontsize(8)

# begin of MT phase
axs[0].axvline(x=init_MT, color="dimgray", ls=":")
axs[1].axvline(x=init_MT, color="dimgray", ls=":")
axs[0].annotate(
    "MT phase",
    xy=(0.72, 5.8),
    bbox=dict(facecolor="lightgray", edgecolor="dimgray", boxstyle="round, pad=0.3"),
)

# legend
leg = axs[1].legend(
    bbox_to_anchor=(0.72, 0.05), title="inclinations", fontsize=8, title_fontsize=9
)
for line, text in zip(leg.get_lines(), leg.get_texts()):
    text.set_color(line.get_color())

# save figure
plt.subplots_adjust(top=0.95, right=0.95)
plt.savefig("../reports/paper/figures/orbital_params_evolution_caseA.pdf", dpi=100)

### - inclination and spin period of each star

In [ ]:
plt.style.use("../config/paper-style.mpl")
fig, axs = plt.subplots(figsize=(4.5, 4), nrows=2, sharex=True)
plt.subplots_adjust(hspace=0)

# tick labels
latex = "{\\rm TAMS}"
axs[-1].set_xlabel(f"$t/t_{latex}$")
# first, orbital period
latex = "{\\rm orb}"
axs[0].set_ylabel("inclination [deg]")
# follows the rot & orb periods
latex_rot = "{\\rm rot}"
latex_orb = "{\\rm orb}"
axs[1].set_ylabel(f"$P_{latex_rot}/P_{latex_orb}$")

for k, inclination in enumerate(inclinations):
    data_null = binary["high_spin_with_ST"][inclination]
    tams_age = t_tams["high_spin_with_ST"][inclination]

    axs[0].plot(
        data_null.data.age / tams_age,
        data_null.data.inclination_1,
        color=colorbar[k],
        label=f"{inclination} deg",
    )

    axs[1].plot(
        data_null.data.age / tams_age,
        data_null.data.P_rot_div_P_orb_1,
        color=colorbar[k],
        label=f"{inclination} deg",
    )

# ticks params & limits
axs[0].set_yticks(np.arange(0, 181, 45))
axs[0].set_ylim([-10, 180])
axs[0].yaxis.set_minor_locator(MultipleLocator(45 / 2))

axs[1].set_yticks(np.arange(0, 2.2, 0.2))
axs[1].set_ylim([0.1, 1.1])
axs[1].yaxis.set_minor_locator(MultipleLocator(0.1))

axs[-1].set_xticks(np.arange(0, 1.1, 0.1))
axs[-1].set_xlim([0, 1])
axs[-1].xaxis.set_minor_locator(MultipleLocator(0.05))

# begin of MT phase
axs[0].axvline(x=init_MT, color="dimgray", ls=":")
axs[1].axvline(x=init_MT, color="dimgray", ls=":")
axs[0].annotate(
    "MT phase",
    xy=(0.72, 150),
    bbox=dict(facecolor="lightgray", edgecolor="dimgray", boxstyle="round, pad=0.3"),
)

# legend
leg = axs[1].legend(
    bbox_to_anchor=(0.72, 0.9), title="inclinations", fontsize=8, title_fontsize=9
)
for line, text in zip(leg.get_lines(), leg.get_texts()):
    text.set_color(line.get_color())

# save figure
plt.subplots_adjust(top=0.95, right=0.95)
plt.savefig("../reports/paper/figures/orbital_params2_evolution_caseA.pdf", dpi=100)

---

## Appendix

### High spin without ST dynamo

#### load data

In [ ]:
root = "../data/raw/asb2022/caseA/high_spin/without_ST"

binary["high_spin_without_ST"] = dict()
companion["high_spin_without_ST"] = dict()
t_tams["high_spin_without_ST"] = dict()
for inclination in inclinations:
    companion["high_spin_without_ST"][inclination] = MesaInfo(
        f"{root}/LOGS_{inclination}/history.data"
    )
    binary["high_spin_without_ST"][inclination] = MesaInfo(
        f"{root}/binary_history_{inclination}.data"
    )
    t_tams["high_spin_without_ST"][inclination] = find_tams_age(
        companion["high_spin_without_ST"][inclination].data.center_h1,
        companion["high_spin_without_ST"][inclination].data.star_age,
    )
    print(f"{root}/LOGS_{inclination}/history.data ... loaded")

In [ ]:
# fig configs
plt.style.use("../config/paper-style.mpl")
fig, axs = plt.subplots(figsize=(4.5, 4), nrows=2, sharex=True)
plt.subplots_adjust(hspace=0)

# second axis config
left, bottom, width, height = [0.22, 0.75, 0.3, 0.17]
ax2 = fig.add_axes([left, bottom, width, height])

# axis labels
latex = "{\\rm TAMS}"
axs[-1].set_xlabel(f"$t/t_{latex}$")
latex = "{\\rm orb}"
axs[0].set_ylabel(f"$P_{latex}$ [days]")
axs[1].set_ylabel("eccentricity")

# plots !
for k, inclination in enumerate(inclinations):
    data_null = binary["high_spin_without_ST"][inclination]
    tams_age = t_tams["high_spin_without_ST"][inclination]

    axs[0].plot(
        data_null.data.age / tams_age, data_null.data.period_days, color=colorbar[k]
    )

    ax2.plot(
        data_null.data.age / tams_age, data_null.data.period_days, color=colorbar[k]
    )

    axs[1].plot(
        data_null.data.age / tams_age,
        data_null.data.eccentricity,
        color=colorbar[k],
        label=f"{inclination} deg",
    )

# ticks params & limits
axs[0].set_yticks(np.arange(2, 15, 1))
axs[0].yaxis.set_minor_locator(MultipleLocator(0.5))
axs[0].set_ylim([1.5, 6.5])
axs[1].set_yticks(np.arange(0, 1.2, 0.1))
axs[1].yaxis.set_minor_locator(MultipleLocator(0.05))
axs[1].set_ylim([0, 0.33])
axs[-1].set_xticks(np.arange(0, 1.1, 0.1))
axs[-1].xaxis.set_minor_locator(MultipleLocator(0.05))
axs[-1].set_xlim([0, 1])
ax2.set_xticks(np.arange(0, 1, 0.1))
ax2.xaxis.set_minor_locator(MultipleLocator(0.05))
ax2.set_xlim([0, 0.5])
ax2.set_yticks(np.arange(2, 4, 0.1))
ax2.yaxis.set_minor_locator(MultipleLocator(0.05))
ax2.set_ylim([2.85, 3.25])

# reduce fontsize for inset plot
for item in (
    [ax2.title, ax2.xaxis.label, ax2.yaxis.label]
    + ax2.get_xticklabels()
    + ax2.get_yticklabels()
):
    item.set_fontsize(8)

# begin of MT phase
axs[0].axvline(x=init_MT, color="dimgray", ls=":")
axs[1].axvline(x=init_MT, color="dimgray", ls=":")
axs[0].annotate(
    "MT phase",
    xy=(0.72, 5.8),
    bbox=dict(facecolor="lightgray", edgecolor="dimgray", boxstyle="round, pad=0.3"),
)

# legend
leg = axs[1].legend(
    bbox_to_anchor=(0.72, 0.05), title="inclinations", fontsize=8, title_fontsize=9
)
for line, text in zip(leg.get_lines(), leg.get_texts()):
    text.set_color(line.get_color())

# save figure
plt.subplots_adjust(top=0.95, right=0.95)
# plt.savefig("../reports/paper/figures/orbital_params_evolution_caseA.pdf", dpi=100)

In [ ]:
plt.style.use("../config/paper-style.mpl")
fig, axs = plt.subplots(figsize=(4.5, 4), nrows=2, sharex=True)
plt.subplots_adjust(hspace=0)

# tick labels
latex = "{\\rm TAMS}"
axs[-1].set_xlabel(f"$t/t_{latex}$")
# first, orbital period
latex = "{\\rm orb}"
axs[0].set_ylabel("inclination [deg]")
# follows the rot & orb periods
latex_rot = "{\\rm rot}"
latex_orb = "{\\rm orb}"
axs[1].set_ylabel(f"$P_{latex_rot}/P_{latex_orb}$")

for k, inclination in enumerate(inclinations):
    data_null = binary["high_spin_without_ST"][inclination]
    tams_age = t_tams["high_spin_without_ST"][inclination]

    axs[0].plot(
        data_null.data.age / tams_age,
        data_null.data.inclination_1,
        color=colorbar[k],
        label=f"{inclination} deg",
    )

    axs[1].plot(
        data_null.data.age / tams_age,
        data_null.data.P_rot_div_P_orb_1,
        color=colorbar[k],
        label=f"{inclination} deg",
    )

# ticks params & limits
axs[0].set_yticks(np.arange(0, 181, 45))
axs[0].set_ylim([-10, 180])
axs[0].yaxis.set_minor_locator(MultipleLocator(45 / 2))

axs[1].set_yticks(np.arange(0, 2.2, 0.2))
axs[1].set_ylim([0.1, 1.1])
axs[1].yaxis.set_minor_locator(MultipleLocator(0.1))

axs[-1].set_xticks(np.arange(0, 1.1, 0.1))
axs[-1].set_xlim([0, 1])
axs[-1].xaxis.set_minor_locator(MultipleLocator(0.05))

# begin of MT phase
axs[0].axvline(x=init_MT, color="dimgray", ls=":")
axs[1].axvline(x=init_MT, color="dimgray", ls=":")
axs[0].annotate(
    "MT phase",
    xy=(0.72, 150),
    bbox=dict(facecolor="lightgray", edgecolor="dimgray", boxstyle="round, pad=0.3"),
)

# legend
leg = axs[1].legend(
    bbox_to_anchor=(0.72, 0.9), title="inclinations", fontsize=8, title_fontsize=9
)
for line, text in zip(leg.get_lines(), leg.get_texts()):
    text.set_color(line.get_color())

# save figure
plt.subplots_adjust(top=0.95, right=0.95)
# plt.savefig("../reports/paper/figures/orbital_params2_evolution_caseA.pdf", dpi=100)

### Low spin with ST dynamo

In [ ]:
root = "../data/raw/asb2022/caseA/low_spin/with_ST"

binary["low_spin_with_ST"] = dict()
companion["low_spin_with_ST"] = dict()
t_tams["low_spin_with_ST"] = dict()
for inclination in inclinations:
    companion["low_spin_with_ST"][inclination] = MesaInfo(
        f"{root}/LOGS_{inclination}/history.data"
    )
    binary["low_spin_with_ST"][inclination] = MesaInfo(
        f"{root}/binary_history_{inclination}.data"
    )
    t_tams["low_spin_with_ST"][inclination] = find_tams_age(
        companion["low_spin_with_ST"][inclination].data.center_h1,
        companion["low_spin_with_ST"][inclination].data.star_age,
    )
    print(f"{root}/LOGS_{inclination}/history.data ... loaded")

In [ ]:
# fig configs
plt.style.use("../config/paper-style.mpl")
fig, axs = plt.subplots(figsize=(4.5, 4), nrows=2, sharex=True)
plt.subplots_adjust(hspace=0)

# second axis config
left, bottom, width, height = [0.22, 0.75, 0.3, 0.17]
ax2 = fig.add_axes([left, bottom, width, height])

# axis labels
latex = "{\\rm TAMS}"
axs[-1].set_xlabel(f"$t/t_{latex}$")
latex = "{\\rm orb}"
axs[0].set_ylabel(f"$P_{latex}$ [days]")
axs[1].set_ylabel("eccentricity")

# plots !
for k, inclination in enumerate(inclinations):
    data_null = binary["low_spin_with_ST"][inclination]
    tams_age = t_tams["low_spin_with_ST"][inclination]

    axs[0].plot(
        data_null.data.age / tams_age, data_null.data.period_days, color=colorbar[k]
    )

    ax2.plot(
        data_null.data.age / tams_age, data_null.data.period_days, color=colorbar[k]
    )

    axs[1].plot(
        data_null.data.age / tams_age,
        data_null.data.eccentricity,
        color=colorbar[k],
        label=f"{inclination} deg",
    )

# ticks params & limits
axs[0].set_yticks(np.arange(2, 15, 1))
axs[0].yaxis.set_minor_locator(MultipleLocator(0.5))
axs[0].set_ylim([1.5, 6.5])
axs[1].set_yticks(np.arange(0, 1.2, 0.1))
axs[1].yaxis.set_minor_locator(MultipleLocator(0.05))
axs[1].set_ylim([0, 0.33])
axs[-1].set_xticks(np.arange(0, 1.1, 0.1))
axs[-1].xaxis.set_minor_locator(MultipleLocator(0.05))
axs[-1].set_xlim([0, 1])
ax2.set_xticks(np.arange(0, 1, 0.1))
ax2.xaxis.set_minor_locator(MultipleLocator(0.05))
ax2.set_xlim([0, 0.5])
ax2.set_yticks(np.arange(2, 4, 0.1))
ax2.yaxis.set_minor_locator(MultipleLocator(0.05))
ax2.set_ylim([2.65, 3.05])

# reduce fontsize for inset plot
for item in (
    [ax2.title, ax2.xaxis.label, ax2.yaxis.label]
    + ax2.get_xticklabels()
    + ax2.get_yticklabels()
):
    item.set_fontsize(8)

# begin of MT phase
axs[0].axvline(x=init_MT, color="dimgray", ls=":")
axs[1].axvline(x=init_MT, color="dimgray", ls=":")
axs[0].annotate(
    "MT phase",
    xy=(0.72, 5.8),
    bbox=dict(facecolor="lightgray", edgecolor="dimgray", boxstyle="round, pad=0.3"),
)

# legend
leg = axs[1].legend(
    bbox_to_anchor=(0.72, 0.05), title="inclinations", fontsize=8, title_fontsize=9
)
for line, text in zip(leg.get_lines(), leg.get_texts()):
    text.set_color(line.get_color())

# save figure
plt.subplots_adjust(top=0.95, right=0.95)
# plt.savefig("../reports/paper/figures/orbital_params_evolution_caseA.pdf", dpi=100)

In [ ]:
plt.style.use("../config/paper-style.mpl")
fig, axs = plt.subplots(figsize=(4.5, 4), nrows=2, sharex=True)
plt.subplots_adjust(hspace=0)

# tick labels
latex = "{\\rm TAMS}"
axs[-1].set_xlabel(f"$t/t_{latex}$")
# first, orbital period
latex = "{\\rm orb}"
axs[0].set_ylabel("inclination [deg]")
# follows the rot & orb periods
latex_rot = "{\\rm rot}"
latex_orb = "{\\rm orb}"
axs[1].set_ylabel(f"$P_{latex_rot}/P_{latex_orb}$")

for k, inclination in enumerate(inclinations):
    data_null = binary["low_spin_with_ST"][inclination]
    tams_age = t_tams["low_spin_with_ST"][inclination]

    axs[0].plot(
        data_null.data.age / tams_age,
        data_null.data.inclination_1,
        color=colorbar[k],
        label=f"{inclination} deg",
    )

    axs[1].plot(
        data_null.data.age / tams_age,
        data_null.data.P_rot_div_P_orb_1,
        color=colorbar[k],
        label=f"{inclination} deg",
    )

# ticks params & limits
axs[0].set_yticks(np.arange(0, 181, 45))
axs[0].set_ylim([-10, 180])
axs[0].yaxis.set_minor_locator(MultipleLocator(45 / 2))

axs[1].set_yticks(np.arange(0, 2.2, 0.2))
axs[1].set_ylim([0.5, 1.7])
axs[1].yaxis.set_minor_locator(MultipleLocator(0.1))

axs[-1].set_xticks(np.arange(0, 1.1, 0.1))
axs[-1].set_xlim([0, 1])
axs[-1].xaxis.set_minor_locator(MultipleLocator(0.05))

# begin of MT phase
axs[0].axvline(x=init_MT, color="dimgray", ls=":")
axs[1].axvline(x=init_MT, color="dimgray", ls=":")
axs[0].annotate(
    "MT phase",
    xy=(0.72, 150),
    bbox=dict(facecolor="lightgray", edgecolor="dimgray", boxstyle="round, pad=0.3"),
)

# legend
leg = axs[0].legend(
    bbox_to_anchor=(0.6, 0.9), title="inclinations", fontsize=8, title_fontsize=9
)
for line, text in zip(leg.get_lines(), leg.get_texts()):
    text.set_color(line.get_color())

# save figure
plt.subplots_adjust(top=0.95, right=0.95)
# plt.savefig("../reports/paper/figures/orbital_params2_evolution_caseA.pdf", dpi=100)

### Low spin without ST dynamo

In [ ]:
root = "../data/raw/asb2022/caseA/low_spin/without_ST"

binary["low_spin_without_ST"] = dict()
companion["low_spin_without_ST"] = dict()
t_tams["low_spin_without_ST"] = dict()
for inclination in inclinations:
    companion["low_spin_without_ST"][inclination] = MesaInfo(
        f"{root}/LOGS_{inclination}/history.data"
    )
    binary["low_spin_without_ST"][inclination] = MesaInfo(
        f"{root}/binary_history_{inclination}.data"
    )
    t_tams["low_spin_without_ST"][inclination] = find_tams_age(
        companion["low_spin_without_ST"][inclination].data.center_h1,
        companion["low_spin_without_ST"][inclination].data.star_age,
    )
    print(f"{root}/LOGS_{inclination}/history.data ... loaded")

In [ ]:
# fig configs
plt.style.use("../config/paper-style.mpl")
fig, axs = plt.subplots(figsize=(4.5, 4), nrows=2, sharex=True)
plt.subplots_adjust(hspace=0)

# second axis config
left, bottom, width, height = [0.22, 0.75, 0.3, 0.17]
ax2 = fig.add_axes([left, bottom, width, height])

# axis labels
latex = "{\\rm TAMS}"
axs[-1].set_xlabel(f"$t/t_{latex}$")
latex = "{\\rm orb}"
axs[0].set_ylabel(f"$P_{latex}$ [days]")
axs[1].set_ylabel("eccentricity")

# plots !
for k, inclination in enumerate(inclinations):
    data_null = binary["low_spin_without_ST"][inclination]
    tams_age = t_tams["low_spin_without_ST"][inclination]

    axs[0].plot(
        data_null.data.age / tams_age, data_null.data.period_days, color=colorbar[k]
    )

    ax2.plot(
        data_null.data.age / tams_age, data_null.data.period_days, color=colorbar[k]
    )

    axs[1].plot(
        data_null.data.age / tams_age,
        data_null.data.eccentricity,
        color=colorbar[k],
        label=f"{inclination} deg",
    )

# ticks params & limits
axs[0].set_yticks(np.arange(2, 15, 1))
axs[0].yaxis.set_minor_locator(MultipleLocator(0.5))
axs[0].set_ylim([1.5, 6.5])
axs[1].set_yticks(np.arange(0, 1.2, 0.1))
axs[1].yaxis.set_minor_locator(MultipleLocator(0.05))
axs[1].set_ylim([0, 0.33])
axs[-1].set_xticks(np.arange(0, 1.1, 0.1))
axs[-1].xaxis.set_minor_locator(MultipleLocator(0.05))
axs[-1].set_xlim([0, 1])
ax2.set_xticks(np.arange(0, 1, 0.1))
ax2.xaxis.set_minor_locator(MultipleLocator(0.05))
ax2.set_xlim([0, 0.5])
ax2.set_yticks(np.arange(2, 4, 0.1))
ax2.yaxis.set_minor_locator(MultipleLocator(0.05))
ax2.set_ylim([2.65, 3.05])

# reduce fontsize for inset plot
for item in (
    [ax2.title, ax2.xaxis.label, ax2.yaxis.label]
    + ax2.get_xticklabels()
    + ax2.get_yticklabels()
):
    item.set_fontsize(8)

# begin of MT phase
axs[0].axvline(x=init_MT, color="dimgray", ls=":")
axs[1].axvline(x=init_MT, color="dimgray", ls=":")
axs[0].annotate(
    "MT phase",
    xy=(0.72, 5.8),
    bbox=dict(facecolor="lightgray", edgecolor="dimgray", boxstyle="round, pad=0.3"),
)

# legend
leg = axs[1].legend(
    bbox_to_anchor=(0.72, 0.05), title="inclinations", fontsize=8, title_fontsize=9
)
for line, text in zip(leg.get_lines(), leg.get_texts()):
    text.set_color(line.get_color())

# save figure
plt.subplots_adjust(top=0.95, right=0.95)
# plt.savefig("../reports/paper/figures/orbital_params_evolution_caseA.pdf", dpi=100)

In [ ]:
plt.style.use("../config/paper-style.mpl")
fig, axs = plt.subplots(figsize=(4.5, 4), nrows=2, sharex=True)
plt.subplots_adjust(hspace=0)

# tick labels
latex = "{\\rm TAMS}"
axs[-1].set_xlabel(f"$t/t_{latex}$")
# first, orbital period
latex = "{\\rm orb}"
axs[0].set_ylabel("inclination [deg]")
# follows the rot & orb periods
latex_rot = "{\\rm rot}"
latex_orb = "{\\rm orb}"
axs[1].set_ylabel(f"$P_{latex_rot}/P_{latex_orb}$")

for k, inclination in enumerate(inclinations):
    data_null = binary["low_spin_without_ST"][inclination]
    tams_age = t_tams["low_spin_without_ST"][inclination]

    axs[0].plot(
        data_null.data.age / tams_age,
        data_null.data.inclination_1,
        color=colorbar[k],
        label=f"{inclination} deg",
    )

    axs[1].plot(
        data_null.data.age / tams_age,
        data_null.data.P_rot_div_P_orb_1,
        color=colorbar[k],
        label=f"{inclination} deg",
    )

# ticks params & limits
axs[0].set_yticks(np.arange(0, 181, 45))
axs[0].set_ylim([-10, 180])
axs[0].yaxis.set_minor_locator(MultipleLocator(45 / 2))

axs[1].set_yticks(np.arange(0, 2.2, 0.2))
axs[1].set_ylim([0.5, 1.7])
axs[1].yaxis.set_minor_locator(MultipleLocator(0.1))

axs[-1].set_xticks(np.arange(0, 1.1, 0.1))
axs[-1].set_xlim([0, 1])
axs[-1].xaxis.set_minor_locator(MultipleLocator(0.05))

# begin of MT phase
axs[0].axvline(x=init_MT, color="dimgray", ls=":")
axs[1].axvline(x=init_MT, color="dimgray", ls=":")
axs[0].annotate(
    "MT phase",
    xy=(0.72, 150),
    bbox=dict(facecolor="lightgray", edgecolor="dimgray", boxstyle="round, pad=0.3"),
)

# legend
leg = axs[0].legend(
    bbox_to_anchor=(0.6, 0.9), title="inclinations", fontsize=8, title_fontsize=9
)
for line, text in zip(leg.get_lines(), leg.get_texts()):
    text.set_color(line.get_color())

# save figure
plt.subplots_adjust(top=0.95, right=0.95)
# plt.savefig("../reports/paper/figures/orbital_params2_evolution_caseA.pdf", dpi=100)

---

### Comparison for a fixed inclination between: high & low spin with & without ST dynamo

In [ ]:
# inclination value chosen
inc = 80

models = {
    "$\\Omega = 0.9$ with ST": "high_spin_with_ST",
    "$\\Omega = 0.9$ without ST": "high_spin_without_ST",
    "$\\Omega = 0.1$ with ST": "low_spin_with_ST",
    "$\\Omega = 0.1$ without ST": "low_spin_without_ST",
}

In [ ]:
# fig configs
plt.style.use("../config/paper-style.mpl")
fig, axs = plt.subplots(figsize=(4.5, 4), nrows=2, sharex=True)
plt.subplots_adjust(hspace=0)

# tick labels
latex = "{\\rm TAMS}"
axs[-1].set_xlabel(f"$t/t_{latex}$")
# first, orbital period
latex = "{\\rm orb}"
axs[0].set_ylabel("inclination [deg]")
# follows the rot & orb periods
latex_rot = "{\\rm rot}"
latex_orb = "{\\rm orb}"
axs[1].set_ylabel(f"$P_{latex_rot}/P_{latex_orb}$")

# plots !
for k, key in enumerate(models):

    data_null = binary[models[key]][inclination]
    tams_age = t_tams[models[key]][inclination]

    ls = "-"
    if k == 1 or k == 3:
        ls = ":"

    axs[0].plot(
        data_null.data.age / tams_age,
        data_null.data.period_days,
        color=colorbar_appendix[k],
        ls=ls,
    )

    axs[1].plot(
        data_null.data.age / tams_age,
        data_null.data.eccentricity,
        color=colorbar_appendix[k],
        ls=ls,
        label=key,
    )

# ticks params & limits
axs[0].set_yticks(np.arange(2, 15, 1))
axs[0].yaxis.set_minor_locator(MultipleLocator(0.5))
axs[0].set_ylim([1.5, 6.5])
axs[1].set_yticks(np.arange(0, 1.2, 0.1))
axs[1].yaxis.set_minor_locator(MultipleLocator(0.05))
axs[1].set_ylim([0, 0.33])
axs[-1].set_xticks(np.arange(0, 1.1, 0.1))
axs[-1].xaxis.set_minor_locator(MultipleLocator(0.05))
axs[-1].set_xlim([0, 1])

# begin of MT phase
axs[0].axvline(x=init_MT, color="dimgray", ls=":")
axs[1].axvline(x=init_MT, color="dimgray", ls=":")
axs[0].annotate(
    "MT phase",
    xy=(0.72, 5.8),
    bbox=dict(facecolor="lightgray", edgecolor="dimgray", boxstyle="round, pad=0.3"),
)

# legend
leg = axs[1].legend(bbox_to_anchor=(0.65, 0.05), fontsize=8, title_fontsize=9)

# save figure
plt.subplots_adjust(top=0.95, right=0.95)
# plt.savefig("../reports/paper/figures/orbital_params_evolution_caseA.pdf", dpi=100)

In [ ]:
# fig configs
plt.style.use("../config/paper-style.mpl")
fig, axs = plt.subplots(figsize=(4.5, 4), nrows=2, sharex=True)
plt.subplots_adjust(hspace=0)

# tick labels
latex = "{\\rm TAMS}"
axs[-1].set_xlabel(f"$t/t_{latex}$")
# first, orbital period
latex = "{\\rm orb}"
axs[0].set_ylabel("inclination [deg]")
# follows the rot & orb periods
latex_rot = "{\\rm rot}"
latex_orb = "{\\rm orb}"
axs[1].set_ylabel(f"$P_{latex_rot}/P_{latex_orb}$")

# plots !
for k, key in enumerate(models):

    data_null = binary[models[key]][inclination]
    tams_age = t_tams[models[key]][inclination]

    ls = "-"
    if k == 1 or k == 3:
        ls = ":"

    axs[0].plot(
        data_null.data.age / tams_age,
        data_null.data.inclination_1,
        color=colorbar_appendix[k],
        ls=ls,
    )

    axs[1].plot(
        data_null.data.age / tams_age,
        data_null.data.P_rot_div_P_orb_1,
        color=colorbar_appendix[k],
        ls=ls,
        label=key,
    )

# ticks params & limits
axs[0].set_yticks(np.arange(0, 181, 45))
axs[0].set_ylim([-10, 180])
axs[0].yaxis.set_minor_locator(MultipleLocator(45 / 2))

axs[1].set_yticks(np.arange(0, 2.2, 0.2))
axs[1].set_ylim([0.5, 1.7])
axs[1].yaxis.set_minor_locator(MultipleLocator(0.1))

axs[-1].set_xticks(np.arange(0, 1.1, 0.1))
axs[-1].set_xlim([0, 1])
axs[-1].xaxis.set_minor_locator(MultipleLocator(0.05))

# begin of MT phase
axs[0].axvline(x=init_MT, color="dimgray", ls=":")
axs[1].axvline(x=init_MT, color="dimgray", ls=":")
axs[0].annotate(
    "MT phase",
    xy=(0.72, 150),
    bbox=dict(facecolor="lightgray", edgecolor="dimgray", boxstyle="round, pad=0.3"),
)

# legend
leg = axs[1].legend(bbox_to_anchor=(0.65, 0.6), fontsize=8, title_fontsize=9)

# save figure
plt.subplots_adjust(top=0.95, right=0.95)
# plt.savefig("../reports/paper/figures/orbital_params_evolution_caseA.pdf", dpi=100)